In [237]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
import re
import pandas as pd


def Getting_Flight_Info(URL):
    driver = webdriver.Chrome('/Users/robjohns/Documents/Metis/Project2/chromedriver')
    driver.get(URL)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    driver.quit()
    data=DataGetter(URL,soup)
    return data



In [239]:
from os import path

URL='https://www.google.com/flights?hl=en#flt=SFO.AUS.2019-08-21;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
data=Getting_Flight_Info(URL)
data=pd.DataFrame(data)
data.to_pickle('Picklefolder/testpickle')


testy=pd.read_pickle('Picklefolder/testpickle')
print(testy)
     

num of flights: 6

                                                  0           1    2    3   \
0  https://www.google.com/flights?hl=en#flt=SFO.A...  2019-08-21  SFO  AUS   
1  https://www.google.com/flights?hl=en#flt=SFO.A...  2019-08-21  SFO  AUS   
2  https://www.google.com/flights?hl=en#flt=SFO.A...  2019-08-21  SFO  AUS   
3  https://www.google.com/flights?hl=en#flt=SFO.A...  2019-08-21  SFO  AUS   
4  https://www.google.com/flights?hl=en#flt=SFO.A...  2019-08-21  SFO  AUS   
5  https://www.google.com/flights?hl=en#flt=SFO.A...  2019-08-21  SFO  AUS   

     4      5        6        7       8       9            10  
0  3.75   79.0   9:20 A   3:05 P  Alaska  AS1218  Airbus A320  
1  3.67   79.0   6:45 P  12:25 A  Alaska  AS1222  Airbus A320  
2  3.58  166.0   7:40 A   1:15 P  United   UA258   Boeing 737  
3  3.67  166.0  10:50 A   4:30 P  United   UA701  Airbus A320  
4  3.55  166.0  11:55 P   5:28 A  United  UA1471  Airbus A319  
5  3.55  199.0   4:45 P  10:18 P  United  UA2358  

In [ ]:
## how to avoid booting from server ideas . . . .
for page in super_short_test_list:
    ### scrape a website
    ### ...
    print(page)
    
    time.sleep(2)
    #time.sleep(.5+2*random.random())
    
import sys
import requests
from bs4 import BeautifulSoup

url = 'http://www.reddit.com'

user_agent = {'User-agent': 'Mozilla/5.0'}
response  = requests.get(url, headers = user_agent)

from fake_useragent import UserAgent

ua = UserAgent()
user_agent = {'User-agent': ua.random}
print(user_agent)

response  = requests.get(url, headers = user_agent)
print(response.text)

In [232]:
##this will take the soupy html file and pull out data values for all flight results from a search##
# looking for [Date,DepartureCity,ArrivalCity,DepartureTime,
#               ArrivalTime,FlightDuration,Price,PlaneType,Airline,FlightNumber]
import re
#'https://www.google.com/flights?hl=en#flt=SFO.AUS.2019-08-21;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
#each page has a different number of flights on it, so for this URL we need to make a list of lists
#that is the length = number of flights on the page#


def DataGetter(URL,soup):

    Locations = re.search('#flt=(.+?);b:1', URL).group(1)
    Spots=Locations.split('.')
    Departure=Spots[0]
    Arrival = Spots[1]
    Date= Spots[2]



    flights=len(soup.find_all('div', attrs={'class': 'gws-flights-results__itinerary-duration'}))
    print('num of flights:',flights)
    BigList = [[] for _ in range(flights)]
    for flight in BigList:
        flight.append(URL)
        flight.append(Date)
        flight.append(Departure)
        flight.append(Arrival)

    # Append flight durations onto each element of the list
    i=-1
    for li in soup.find_all('div', attrs={'class': 'gws-flights-results__duration flt-subhead1Normal'}):
        i+=1
        chonk=li.get_text()
        try:
            ch=chonk.split('h')
            onk=chonk.split(' ')
            num2=onk[1]
            flight_duration=float(ch[0])+(float(num2[:-1])/60)
            flight_duration = round(flight_duration,2)
            BigList[i].append(flight_duration)
            #print ('flight_duration:',flight_duration)
        except:
            BigList[i].append('NaN')
            print('There was an issue')



    #for li in soup.find_all('div', attrs={'class': 'gws-flights-results__price'}):
        #print(li)


    #here we add more flight info from the jstcache. . . beware this number seems to change from day to day
    i=-1
    for li in soup.find_all('jsl', attrs={'jstcache': "8836"}):
        i+=1
        text = li.get_text()

          #finding price
        try:
            found = re.search('From (.+?)Trip', text).group(1)
            dollar=found.replace(',','')
            found=float(dollar[1:-1])
        except AttributeError:
        # if the sentence structure i didnt expect show up
            found = 'NaN' # apply your error handling
        #print('cost:',found)
        BigList[i].append(found)
        #finding departure and arrival times
        try:
            found = re.search('Departure time: (.+?)M', text).group(1)
            found2 = re.search('Arrival time: (.+?)M', text).group(1)

            #departure=found.replace(',','')
            #found=float(dollar[1:-1])
        except AttributeError:
        # if the sentence structure i didnt expect show up
            found = 'NaN' # apply your error handling
        #print('Departure Time:',found)
        #print('Arrival Time:',found2)
        BigList[i].append(found)
        BigList[i].append(found2)


        #grab airline
        try:
            found = re.search('by (.+?).Depa', text).group(1)

            #departure=found.replace(',','')
            #found=float(dollar[1:-1])
        except AttributeError:
        # if the sentence structure i didnt expect show up
            found = 'NaN' # apply your error handling
        #print('Airline:',found)
        BigList[i].append(found)
    #print('')
        #grab PlaneType and flight number
    i=-1    
    for li in soup.find_all('div', attrs={'class': 'gws-flights-results__other-leg-info'}):
            i+=1
            #print(li.get_text())
            st=li.get_text()
            #st=str(st.text())
            result = re.match("(?P<PlaneType>[A-Za-z]+.[A-Za-z]*[0-9]+)(?P<FlightNum>[A-Za-z]+\s[0-9]+)", st)
            if result:
                Planetype=result.group('PlaneType')
                Flightnumber=result.group('FlightNum')
                Flightnumber=Flightnumber.replace('\xa0','')
                #print('Flightnum:',Flightnumber)
                #print('Planetype:',Planetype)
                BigList[i].append(Flightnumber)
                BigList[i].append(Planetype)
                #print("")
            else:
                errormsg=('REGEXerror:',st)
                BigList[i].append(errormsg)
                BigList[i].append(errormsg)

    return BigList

In [231]:
for lists in BigList:
    print(len(lists))

11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11


In [207]:
##Format URLs, can create these lists:      super_short_test_list//test_urls//full_cities_lessdates_urls//
##//full_dates_lesscities_urls//full_url_list//come_home_URLS##

cities=['SFO','AUS','ATL','BOI','BOS','CUN','ORD','DEN','DTW','FAI','JFK','SAN']

##links look like this for one ways,nonstop, with overhead bags allowed, and frontier airlines excluded##
'https://www.google.com/flights?hl=en#flt=SFO.AUS.2019-08-21;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'

#This makes all the dates from july 22nd to the end of the year, will update from date time later if i have time
dates=['2019-07-22']
for days in range(23,32):
    date = '2019-07-' + str(days)
    dates.append(date)
for days in range(1,32):
    if days>=10:
        date = '2019-08-' + str(days)
        dates.append(date)
    else:
        date = '2019-08-0' + str(days)
        dates.append(date)
for days in range(1,31):
    if days>=10:
        date = '2019-09-' + str(days)
        dates.append(date)
    else:
        date = '2019-09-0' + str(days)
        dates.append(date)
for days in range(1,32):
    if days>=10:
        date = '2019-10-' + str(days)
        dates.append(date)
    else:
        date = '2019-10-0' + str(days)
        dates.append(date) 

for days in range(1,31):
    if days>=10:
        date = '2019-11-' + str(days)
        dates.append(date)
    else:
        date = '2019-11-0' + str(days)
        dates.append(date)           

for days in range(1,32):
    if days>=10:
        date = '2019-12-' + str(days)
        dates.append(date)
    else:
        date = '2019-12-0' + str(days)
        dates.append(date)         

## This makes lists of URLS to use generating data##
super_short_test_list=[]
super_short_test_list_dates=[]
for city in cities[::6]:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates[::43]:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        super_short_test_list.append(URL)
        super_short_test_list_dates.append(date)
        
print('super short test set has this many values:'+str(len(super_short_test_list)))

test_urls=[]
for city in cities[::5]:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates[::29]:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        test_urls.append(URL)
print('test set has this many values:'+str(len(test_urls)))        
        
full_cities_lessdates_urls=[]
for city in cities:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates[::15]:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        full_cities_lessdates_urls.append(URL)
print('full city set has this many values:'+str(len(full_cities_lessdates_urls)))   

full_dates_lesscities_urls=[]
for city in cities[::3]:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        full_dates_lesscities_urls.append(URL)
print('full date set has this many values:'+str(len(full_dates_lesscities_urls)))  

full_url_list=[]
for city in cities:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        full_url_list.append(URL)
print('full set has this many values:'+str(len(full_url_list)))  

come_home_URLS=[]
for city in cities:
    URLc='https://www.google.com/flights?hl=en#flt='+city+'.'+'SEA.'
    for date in dates:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        come_home_URLS.append(URL)
print('come home set has this many values:'+str(len(come_home_URLS)))  



super short test set has this many values:8
test set has this many values:18
full city set has this many values:132
full date set has this many values:652
full set has this many values:1956
come home set has this many values:1956
